In [166]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

# New Section

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [167]:
spark.sparkContext.addFile(url)


In [169]:
read_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
read_df.show(5)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [170]:
read_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)



In [171]:
# 2. Create a temporary view of the DataFrame.

tem_table=read_df.createOrReplaceTempView('home_sales')



In [200]:
spark.sql(
    """
    select  extract(year from date) AS year,
    round(AVG(price), 2) AS average_price
    from home_sales
    where bedrooms = 4
    group by  year
    order by year
    """
 ).show()


+----+-------------+
|year|average_price|
+----+-------------+
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [204]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
sql="""(
    select
    date_built as Year,
    round (AVG(price),2 ) as Avg_Price
    from home_sales
    where bedrooms =3 and bathrooms =3
    group by date_built
    order by date_built DESC
    )
    """
spark.sql(sql).show()




+----+---------+
|Year|Avg_Price|
+----+---------+
|2017|292676.79|
|2016|290555.07|
|2015| 288770.3|
|2014|290852.27|
|2013|295962.27|
|2012|293683.19|
|2011|291117.47|
|2010|292859.62|
+----+---------+



In [205]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
sql_1="""(

   select date_built as Year_built,
   round(AVG(price),2) as Avg_Price
   from home_sales
   WHERE bedrooms =3
   AND bathrooms =3
   AND floors = 2
   AND sqft_living >= 2000
   group by date_built
   order by date_built DESC
   )
  """
spark.sql(sql_1).show()



+----------+---------+
|Year_built|Avg_Price|
+----------+---------+
|      2017|280317.58|
|      2016| 293965.1|
|      2015|297609.97|
|      2014|298264.72|
|      2013|303676.79|
|      2012|307539.97|
|      2011|276553.81|
|      2010|285010.22|
+----------+---------+



In [207]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

sql_1 ="""

  select view,
  round (AVG(price),2) as AVG_Price
  from home_sales
  group BY view
  having AVG(price) >= 350000
  order BY view desc
  """
spark.sql(sql_1).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| AVG_Price|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
|  80| 991767.38|
+----+----------+
only showing top 20 rows

--- 0.37317395210266113 seconds ---


In [176]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")


DataFrame[]

In [177]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [178]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()
sql_2 = """

  select view,
  round (AVG(price),2) as Avg_Price
  from home_sales
  group by view
  having round(AVG(price),2) >= 350000
  order by view desc

  """
spark.sql(sql_2).show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view| Avg_Price|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
|  80| 991767.38|
+----+----------+
only showing top 20 rows

--- 0.4730212688446045 seconds ---


In [181]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
read_df.write.partitionBy('date_built').mode ('overwrite').parquet('home_sales_parquet')


In [182]:
 type(read_df)

pyspark.sql.dataframe.DataFrame

In [186]:
# 11. Read the parquet formatted data.
parque_home_sales_df= spark.read.parquet('home_sales_parquet')


In [189]:
# 12. Create a temporary table for the parquet data.
parque_home_sale_df.createOrReplaceTempView("temp_table_home_sales_parquet")

In [208]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

sql_sql="""
  select view,
  round (AVG(price),2) as Avg_Price
  from home_sales
  group by view
  having round(AVG(price),2) >= 350000
  order by view desc
  """
spark.sql(sql_sql).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| Avg_Price|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
|  80| 991767.38|
+----+----------+
only showing top 20 rows

--- 0.3423621654510498 seconds ---


In [197]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table home_sales')


DataFrame[]

In [198]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')



False